In [1]:
import os
import sys

from dotenv import load_dotenv

sys.path.append(os.path.abspath(os.pardir))
load_dotenv()

True

In [2]:
from langchain.chains import LLMCheckerChain
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro", max_output_tokens=800)

In [6]:
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

prompt = PromptTemplate.from_template("Summarize this text: {text}?")

text = """
A Bellman equation, named after Richard E. Bellman, is a necessary condition for optimality associated with the mathematical optimization method known as dynamic programming. It writes the "value" of a decision problem at a certain point in time in terms of the payoff from some initial choices and the "value" of the remaining decision problem that results from those initial choices. This breaks a dynamic optimization problem into a sequence of simpler subproblems, as Bellman's “principle of optimality" prescribes. The equation applies to algebraic structures with a total ordering; for algebraic structures with a partial ordering, the generic Bellman's equation can be used.

The Bellman equation was first applied to engineering control theory and to other topics in applied mathematics, and subsequently became an important tool in economic theory; though the basic concepts of dynamic programming are prefigured in John von Neumann and Oskar Morgenstern's Theory of Games and Economic Behavior and Abraham Wald's sequential analysis.[citation needed] The term 'Bellman equation' usually refers to the dynamic programming equation associated with discrete-time optimization problems. In continuous-time optimization problems, the analogous equation is a partial differential equation that is called the Hamilton–Jacobi–Bellman equation.

In discrete time any multi-stage optimization problem can be solved by analyzing the appropriate Bellman equation. The appropriate Bellman equation can be found by introducing new state variables (state augmentation). However, the resulting augmented-state multi-stage optimization problem has a higher dimensional state space than the original multi-stage optimization problem - an issue that can potentially render the augmented problem intractable due to the “curse of dimensionality”. Alternatively, it has been shown that if the cost function of the multi-stage optimization problem satisfies a "backward separable" structure, then the appropriate Bellman equation can be found without state augmentation.
"""

chain = prompt | llm | StrOutputParser()
summary = chain.invoke({"text": text})
print(summary)

- A Bellman equation is a mathematical tool used in dynamic programming to find the optimal solution to a decision problem.
- It breaks down the problem into a sequence of simpler subproblems and expresses the value of a decision at a certain time in terms of the payoff from initial choices and the value of the remaining problem.
- The Bellman equation is widely used in engineering control theory, economic theory, and other applied mathematics.
- In discrete-time optimization problems, it is a difference equation, while in continuous-time problems, it is a partial differential equation called the Hamilton-Jacobi-Bellman equation.
- Finding the appropriate Bellman equation for a multi-stage optimization problem can be challenging, and state augmentation may be needed, potentially leading to computational difficulties.
- However, if the cost function has a "backward separable" structure, the Bellman equation can be found without state augmentation.


### Chain of density


In [14]:
cod_template = """Article: {text}
You will generate increasingly concise, entity-dense summaries of the above article.
Repeat the following 2 steps 5 times.
Step 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary.
Step 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities.
A missing entity is:
- relevant to the main story,
- specific yet concise (5 words or fewer),
- novel (not in the previous summary),
- faithful (present in the article),
- anywhere (can be located anywhere in the article).
Guidelines:
- The first summary should be long (4-5 sentences, ~80 words) yet highly non-specific, containing little information beyond the entities marked as missing. Use overly verbose language and fillers (e.g., "this article discusses") to reach ~80 words.
- Make every word count: rewrite the previous summary to improve flow and make space for additional entities.
- Make space with fusion, compression, and removal of uninformative phrases like "the article discusses".
- The summaries should become highly dense and concise yet self-contained, i.e., easily understood without the article.
- Missing entities can appear anywhere in the new summary.
- Never drop entities from the previous summary. If space cannot be made, add fewer new entities.
Remember, use the exact same number of words for each summary.
{format_instructions}
"""

Docs: [JsonOutputParser](https://python.langchain.com/docs/modules/model_io/output_parsers/types/json)


In [22]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel


class COD(BaseModel):
    missing_entities: str
    denser_summary: str


class COD_List(BaseModel):
    data: list[COD]


parser = JsonOutputParser(pydantic_object=COD_List)

cod_prompt = PromptTemplate(
    template=cod_template,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = cod_prompt | llm | parser
result = chain.invoke({"text": text})
result['data']

[{'missing_entities': 'Richard E. Bellman, Dynamic programming',
  'denser_summary': "Richard E. Bellman's Bellman equation is a crucial optimization tool in dynamic programming. It allows breaking a dynamic optimization problem into simpler subproblems, as prescribed by Bellman's principle of optimality."},
 {'missing_entities': 'Algebraic structures, Total ordering, Partial ordering',
  'denser_summary': 'The Bellman equation applies to algebraic structures with total or partial ordering. In discrete time, multi-stage optimization problems can be solved by analyzing the appropriate Bellman equation.'},
 {'missing_entities': 'Continuous-time optimization, Hamilton–Jacobi–Bellman equation',
  'denser_summary': 'In continuous-time optimization, the Hamilton–Jacobi–Bellman equation is analogous to the Bellman equation for discrete time.'},
 {'missing_entities': 'State space, State augmentation, Curse of dimensionality',
  'denser_summary': 'State augmentation can be used to find the appr